In [1]:
import requests
import pickle
import copy
import datetime as dt

In [2]:
verbose = True

In [3]:
# This class maps a dictionary into a simple Python object
class Dict2Obj(object):
    '''
    Class to transform a dict into an obj.
    
    Dict keys must be string which becomes the obj attributes' names.
    '''
    
    def __init__(self, dic):
        for key in dic:
            setattr(self, key, dic[key])
        else:
            return None
    def __repr__(self):
        return str(self.__dict__)
    def __copy__(self):
        return Dict2Obj(self.__dict__)

In [4]:
# Please launch the server before continuing
url_exploration = 'http://localhost:5044/api/exploration'
url_function_request = 'http://localhost:5044/api/function_request'

In [5]:
def generic_function(submodule_name, file_name, function_name,
                     method_name=None, **args):
    '''
    Post a request to the server asking to execute
    the corresponding remote function.
    
    :param str submodule_name: the name of the submodule
        which contains the function to be executed

    :param str file_name: the name of the file
        which contains the function to be executed

    :param str function_name: the name of the function
        to be executed

    :param str method_name: if this parameter is not None
        it means function_name is actually the name of a class ;
        thus method_name contains the name of the method of the class
        to be executed.

    :param dict **args: is a dict of arguments for the function
        we want to execute. Besides these arguments, it may also
    contains:
        - a variable _dic, which contains the object's __dict__
            and which is passed through the API
        - a variable _obj, which contains the object itself.
            It cannot be passed through the API, but it is used to
            locally modify the object after the result has been
            returned by the API.

    :return: result from the remote function.

    :raise: Exception if the POST request returns an error.
    '''

    args = dict((k, v) for (k, v) in args.items() if v != '__$__')
    
    # for special methods (such as __repr__), the obj is passed in _dic,
    # so we need to pop it out and replace it by its __dict__
    if method_name is not None and method_name[:2] == '__':
        obj = args['_dic']
        args['_dic'] = dict((k, v) for (k, v) in args['_dic'].__dict__.items()
                            if not hasattr(v, "__call__"))
    else:    
        obj = args.pop("_obj") if "_obj" in args else None

    dic = {'module':'main_module.' + submodule_name + '.' + file_name,
           'function':function_name, 'args':args}

    if method_name:
        dic['method'] = method_name 
    req = requests.post(url_function_request, data=pickle.dumps(dic))
    incoming = pickle.loads(req.content)

    if req.status_code == 400 or req.status_code == 521:
        print(req.json()['error'])
        raise Exception(req.json()['error'])
    
    if req.status_code == 201:
        if verbose:
            print('creating class dynamically...')
        # populate instance with its __dict__
        instance = Dict2Obj(incoming['_dic'])
        methods = incoming['methods']
        # populate instance with its methods
        for method_name, list_of_args in methods.items():
            # we don't want to have access to the __init__ here.
            if method_name != '__init__':
                list_of_args.insert(0, "_dic")
                function = dic_to_func(submodule_name, file_name, function_name,
                                       list_of_args, method_name=method_name)
                setattr(instance, method_name, function)
        return change_instance_functions_to_pass_obj_and_dic(instance,
                                                             function_name)
    elif req.status_code == 202:
        if verbose:
            print('updating object...')
        method_result = incoming['method_result']
        # updating the instance __dict__
        obj.__dict__.update(incoming['_dic'])
        return method_result
    
    else:        
        return incoming


In [6]:
def dic_to_func(submodule_name, file_name, function_name, list_of_args,
                method_name=None):
    '''String manipulations to create lambda function from its signature.'''

    s = 'function = lambda '
    for attr in [x for x in list_of_args if x != '**kwargs']:
        s += attr + ','
    s += '**kwargs: generic_function("' + '", "'.join([submodule_name,
                                                       file_name,
                                                       function_name])
    if method_name:
        s += '", "' + method_name
    s += '", **{'
    for attr_name in [x.split('=')[0] for x in list_of_args
                        if x != '**kwargs']:
        s += '"' + attr_name + '":' + attr_name + ','
    s = s[:-1] + "}, **kwargs)"
    print(s)
    exec(s)
    return locals()['function']

In [7]:
def change_instance_functions_to_pass_obj_and_dic(instance, function_name):
    '''String manipulations to pass the _obj and the _dic arguments
    to the lambda function without the user having to worry about it.'''

    if verbose:
        print('in change_instance_functions_to_pass_obj_and_dic')
    
    global _dict_of_variables
    try:
        _dict_of_variables
    except:
        _dict_of_variables = {}
  #  if "_dict_of_variables" not in globals():
  #      globals()['_dict_of_variables'] = {}
    
    copy_of_instance = copy.copy(instance)
    
    tag = str(dt.datetime.now())
    _dict_of_variables[tag + '-0'] = instance
    _dict_of_variables[tag + '-1'] = copy_of_instance
    counter = 1
    
    dic_special_method = {}
    
    for name, func in instance.__dict__.items():
        if hasattr(func, '__call__'):
            counter += 1
            _dict_of_variables[tag + '-' + str(counter)] = name
            
            s = 'instance.temp = lambda '
            for attr in [x for x in list(func.__code__.co_varnames)
                         if x not in ['_dic', 'kwargs']]:
                s += attr + '="__$__",'
            s += '**kwargs: getattr(_dict_of_variables["' + tag + \
                 '-0"], _dict_of_variables["' + tag + '-' + str(counter) + '"])'
            s += '(_obj = _dict_of_variables["' + tag + \
                 '-1"], _dic=dict((k, v) for (k, v) in _dict_of_variables["' + \
                 tag + '-1"].__dict__.items() if not hasattr(v, "__call__")),**{'
            for attr_name in [x.split('=')[0] for x in list(func.__code__.co_varnames)
                              if x not in ['_dic', 'kwargs']]:
                s += '"' + attr_name + '":' + attr_name + ','
            if s[-1] == ',':
                s = s[:-1]
            s += '}, **kwargs)'
            print(s)
            exec(s)
            if name[:2] != '__':
                setattr(copy_of_instance, name, instance.temp)
            else:
                # special methods such as __repr__ are stacked
                # in dic_special_method, which is passed
                # to the class and not to the instance
                dic_special_method[name] = func
                del copy_of_instance.__dict__[name]
            delattr(instance, 'temp')
            
    copy_of_instance.__class__ = type(function_name, (Dict2Obj,), dic_special_method)
    return copy_of_instance

In [8]:
# here we request for the JSON describing the remote file structure.
# The whole code is based on a precise file structure.
# See remote module for an example.
# It could probably be generalized even further
# but that's not worth the trouble for now.

r = requests.get(url_exploration)
print("status code: " + str(r.status_code))
print(r.json())

status code: 200
{'submodule1': {'file1': {'Hallo': {'__init__': ['name'], '__repr__': [], 'polite': ['toto', 't=3', '**kwargs'], 'test': []}, 'bonjour': ['x'], 'hello': ['y', 'x', 'z=1', '**kwargs']}, 'file2': {'goodbye': ['x', 'y', 'z']}}, 'submodule2': {'file3': {'bonjour': ['x'], 'hello': ['y', 'x', 'z=1', '**kwargs']}, 'file4': {'goodbye': ['x', 'y', 'z']}}}


In [9]:
def create_class_from_name(submodule_name):
    '''Instanciate and return an object that reproduces
    the structure of a remote submodule.'''

    r = requests.get(url_exploration)
    dic_files = r.json()[submodule_name]
    for file_name, dic_function_names in dic_files.items():
        for function_name, list_of_args in dic_function_names.items():
            if type(list_of_args) == dict:
                list_of_args = list_of_args["__init__"]
            dic_files[file_name][function_name] = dic_to_func(submodule_name,
                                                              file_name,
                                                              function_name,
                                                              list_of_args)
    obj = Dict2Obj(dic_files)
    for k, v in obj.__dict__.items():
        setattr(obj, k, Dict2Obj(v))
    return obj

In [10]:
# here we create local proxy for distant submodule1
submodule1 = create_class_from_name('submodule1')

function = lambda name,**kwargs: generic_function("submodule1", "file1", "Hallo", **{"name":name}, **kwargs)
function = lambda x,**kwargs: generic_function("submodule1", "file1", "bonjour", **{"x":x}, **kwargs)
function = lambda y,x,z=1,**kwargs: generic_function("submodule1", "file1", "hello", **{"y":y,"x":x,"z":z}, **kwargs)
function = lambda x,y,z,**kwargs: generic_function("submodule1", "file2", "goodbye", **{"x":x,"y":y,"z":z}, **kwargs)


In [11]:
# we run one function of the submodule
submodule1.file1.hello(3, 4, s=5)

13

In [12]:
# we instanciate a class of the submodule
x = submodule1.file1.Hallo(name="Pierre")

creating class dynamically...
function = lambda _dic,**kwargs: generic_function("submodule1", "file1", "Hallo", "__repr__", **{"_dic":_dic}, **kwargs)
function = lambda _dic,toto,t=3,**kwargs: generic_function("submodule1", "file1", "Hallo", "polite", **{"_dic":_dic,"toto":toto,"t":t}, **kwargs)
function = lambda _dic,**kwargs: generic_function("submodule1", "file1", "Hallo", "test", **{"_dic":_dic}, **kwargs)
in change_instance_functions_to_pass_obj_and_dic
instance.temp = lambda **kwargs: getattr(_dict_of_variables["2017-09-03 22:48:46.382459-0"], _dict_of_variables["2017-09-03 22:48:46.382459-2"])(_obj = _dict_of_variables["2017-09-03 22:48:46.382459-1"], _dic=dict((k, v) for (k, v) in _dict_of_variables["2017-09-03 22:48:46.382459-1"].__dict__.items() if not hasattr(v, "__call__")),**{}, **kwargs)
instance.temp = lambda toto="__$__",t="__$__",**kwargs: getattr(_dict_of_variables["2017-09-03 22:48:46.382459-0"], _dict_of_variables["2017-09-03 22:48:46.382459-3"])(_obj = _dict_of_var

In [13]:
# the type is the good one
type(x)

__main__.Hallo

In [14]:
x.__dict__

{'name': 'Pierre',
 'polite': <function __main__.<lambda>>,
 'sentence': 'Hallo Pierre !',
 'test': <function __main__.<lambda>>}

In [15]:
# the object has both attributes and methods
# (to be precise, it has no method, but rather functions)
x

updating object...


My name is Pierre

In [16]:
x.name = 'Paul'

In [17]:
x.__dict__

{'name': 'Paul',
 'polite': <function __main__.<lambda>>,
 'sentence': 'Hallo Pierre !',
 'test': <function __main__.<lambda>>}

In [18]:
x

updating object...


My name is Paul

In [19]:
# below lies the magic ! We can call remote methods on objects
# and they will be executed almost as usual.
# The method can do two things
# (and in this example the method 'polite' actually does these two things) : 
# - return a result (which is in the return statement in the remote method)
# - modify in-place the object calling the method (here the 'sentence' attribute is modified)
# The only restriction is that the methods of the object cannot be modified in-place, for now. 
# But we could imagine a solution to this problem
# (which is quite far-fetched anyway) by using some kind of flag in the server.  

# Two delicate things are handled behind the scenes :
# - the **kwargs are working ('s' is a kwarg here)
# - the by-default arguments are working as well
#   (if you look at polite, you will see it has a by-default 't' argument)

res = x.polite(toto='Smith', s=2)

updating object...


In [20]:
res

'Welcome to my nice hotel, Mr Smith'

In [21]:
x

updating object...


My name is Paul

In [22]:
res = x.polite(toto='42')
print(res)

updating object...
Welcome to my nice hotel, Mr 42


In [23]:
# another example, without all the intermediate steps. 
# The only weird thing compared to what we could expect is the [1].
# All the rest is normal.
# This line requires two requests :
# - one to instantiate the object of the class Hallo()
# - one to execute the method 'polite'
obj = submodule1.file1.Hallo(name='Augustin Louis')
print(obj.polite('Cauchy'))
obj.sentence

creating class dynamically...
function = lambda _dic,**kwargs: generic_function("submodule1", "file1", "Hallo", "__repr__", **{"_dic":_dic}, **kwargs)
function = lambda _dic,toto,t=3,**kwargs: generic_function("submodule1", "file1", "Hallo", "polite", **{"_dic":_dic,"toto":toto,"t":t}, **kwargs)
function = lambda _dic,**kwargs: generic_function("submodule1", "file1", "Hallo", "test", **{"_dic":_dic}, **kwargs)
in change_instance_functions_to_pass_obj_and_dic
instance.temp = lambda **kwargs: getattr(_dict_of_variables["2017-09-03 22:48:46.504694-0"], _dict_of_variables["2017-09-03 22:48:46.504694-2"])(_obj = _dict_of_variables["2017-09-03 22:48:46.504694-1"], _dic=dict((k, v) for (k, v) in _dict_of_variables["2017-09-03 22:48:46.504694-1"].__dict__.items() if not hasattr(v, "__call__")),**{}, **kwargs)
instance.temp = lambda toto="__$__",t="__$__",**kwargs: getattr(_dict_of_variables["2017-09-03 22:48:46.504694-0"], _dict_of_variables["2017-09-03 22:48:46.504694-3"])(_obj = _dict_of_var

'Hallo Augustin Louis Cauchy'

In [24]:
_dict_of_variables

updating object...
updating object...


{'2017-09-03 22:48:46.382459-0': {'name': 'Pierre', 'sentence': 'Hallo Pierre !', '__repr__': <function <lambda> at 0x117085e18>, 'polite': <function <lambda> at 0x1170ea048>, 'test': <function <lambda> at 0x1170ea1e0>},
 '2017-09-03 22:48:46.382459-1': My name is Paul,
 '2017-09-03 22:48:46.382459-2': '__repr__',
 '2017-09-03 22:48:46.382459-3': 'polite',
 '2017-09-03 22:48:46.382459-4': 'test',
 '2017-09-03 22:48:46.504694-0': {'name': 'Augustin Louis', 'sentence': 'Hallo Augustin Louis !', '__repr__': <function <lambda> at 0x1170ea510>, 'polite': <function <lambda> at 0x1170ea488>, 'test': <function <lambda> at 0x1170ea950>},
 '2017-09-03 22:48:46.504694-1': My name is Augustin Louis,
 '2017-09-03 22:48:46.504694-2': '__repr__',
 '2017-09-03 22:48:46.504694-3': 'polite',
 '2017-09-03 22:48:46.504694-4': 'test'}